<a href="https://colab.research.google.com/github/ericatsu/msc_capsmodel/blob/main/caps_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import os

In [ ]:
# Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Define constants
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 10
TRAIN_DIR = "/content/drive/MyDrive/msc_capsmodel/dataset/training"
TEST_DIR = "/content/drive/MyDrive/msc_capsmodel/dataset/testing"

Mounted at /content/drive


In [ ]:
# Data loading and preprocessing with augmentation
def load_data(data_dir, is_training=True):
    if is_training:
        datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            zoom_range=0.2,
            brightness_range=[0.8, 1.2],
        )
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

train_data = load_data(TRAIN_DIR, is_training=True)
test_data = load_data(TEST_DIR, is_training=False)

Found 5711 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
# Inspect a batch of data
for data_batch, labels_batch in train_data:
    print(data_batch.shape)
    print(labels_batch.shape)
    break

(32, 128, 128, 3)
(32, 4)


In [ ]:
# Squash activation function for capsules
def squash(vectors, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + 1e-8)
    return scale * vectors

In [ ]:
# Capsule layer
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings

    def build(self, input_shape):
        self.W = self.add_weight(shape=[1, input_shape[1], self.num_capsules, self.dim_capsules, input_shape[-1]],
                                 initializer='glorot_uniform',
                                 name='W')

    def call(self, inputs):
        u = tf.expand_dims(inputs, 2)
        u = tf.expand_dims(u, 3)
        u_hat = tf.reduce_sum(self.W * u, axis=-1)

        b = tf.zeros(shape=[tf.shape(inputs)[0], inputs.shape[1], self.num_capsules, 1])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            s = tf.reduce_sum(c * u_hat, axis=1, keepdims=True)
            v = squash(s, axis=-1)
            if i < self.routings - 1:
                b += tf.reduce_sum(u_hat * v, axis=-1, keepdims=True)

        return tf.squeeze(v, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.dim_capsules)

In [ ]:
# Custom loss function for capsule network
def margin_loss(y_true, y_pred):
    num_classes = tf.shape(y_pred)[-1]
    y_true = tf.one_hot(tf.argmax(y_true, axis=1), depth=num_classes)
    m_plus = 0.9
    m_minus = 0.1
    lambda_val = 0.5

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.0 - 1e-9)

    L = y_true * tf.square(tf.maximum(0., m_plus - y_pred)) + \
        lambda_val * (1 - y_true) * tf.square(tf.maximum(0., y_pred - m_minus))

    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

# Create the hybrid CNN-Capsule model
def create_hybrid_model():
    # Load pre-trained ResNet50
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Unfreeze the last few layers of ResNet50
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Add custom layers on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = tf.keras.layers.Reshape((-1, 256))(x)
    capsule = CapsuleLayer(num_capsules=NUM_CLASSES, dim_capsules=16)(x)
    outputs = tf.keras.layers.Lambda(lambda z: tf.sqrt(tf.reduce_sum(tf.square(z), axis=-1)))(capsule)
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

In [ ]:
# Create and compile the model
model = create_hybrid_model()
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss=margin_loss, metrics=['accuracy'])

# Print model summary
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 134, 134, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 64, 64, 64)     │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 64, 64, 64)     │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 64, 64, 64)     │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 66, 66, 64)     │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 32, 32, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 32, 32, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 32, 32, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 32, 32, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 32, 32, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 32, 32, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 32, 32, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 32, 32, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 32, 32, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 24,129,664 (92.05 MB)

 Trainable params: 24,076,032 (91.84 MB)

 Non-trainable params: 53,632 (209.50 KB)

In [ ]:
# Learning rate scheduler
def lr_schedule(epoch):
    lr = 1e-4
    if epoch > 10:
        lr *= 0.1
    if epoch > 20:
        lr *= 0.1
    return lr

In [ ]:
# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7)

In [11]:
# Train the model
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), early_stopping, reduce_lr]
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


179/179 ━━━━━━━━━━━━━━━━━━━━ 4328s 23s/step - accuracy: 0.4187 - loss: 0.5593 - val_accuracy: 0.1747 - val_loss: 0.5875 - learning_rate: 1.0000e-04
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1898s 11s/step - accuracy: 0.8169 - loss: 0.1693 - val_accuracy: 0.2082 - val_loss: 0.6222 - learning_rate: 1.0000e-04
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1995s 11s/step - accuracy: 0.8722 - loss: 0.1172 - val_accuracy: 0.2845 - val_loss: 0.5898 - learning_rate: 1.0000e-04
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1899s 11s/step - accuracy: 0.8900 - loss: 0.1027 - val_accuracy: 0.3402 - val_loss: 0.6589 - learning_rate: 1.0000e-04
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 2005s 11s/step - accuracy: 0.9123 - loss: 0.0826 - val_accuracy: 0.7414 - val_loss: 0.2477 - learning_rate: 1.0000e-04
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1950s 11s/step - accuracy: 0.9140 - loss: 0.0775 - val_accuracy: 0.8345 - val_loss: 0.1600 - learning_rate: 1.0000e-04
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 1984s 11s/step - 

In [12]:
model.save('/content/drive/MyDrive/msc_capsmodel/saved_model/capsModelV1.h5')

In [13]:
model.save('/content/drive/MyDrive/msc_capsmodel/saved_model/capsModelV1.keras')

In [14]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.4f}")

41/41 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.9096 - loss: 0.0882
Test accuracy: 0.9008
